# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from azureml.core import Model # Used to get model information
from azureml.data.dataset_factory import TabularDatasetFactory

print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-project'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AM8VS8HQQ to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [ ]:
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "udacity-1"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

I am using a Gaming dataset found on Kaggle. The dataset consists of 40 columns including "GameId" which is the unique identifier and "BlueWins" which indicates if Blue wins or Red.
I will be performing a binary classification as there are only two possible outcomes.

In [3]:
data_loc = "https://raw.githubusercontent.com/AishuDas/nd00333-capstone/master/starter_file/high_diamond_ranked_10min.csv"
ds = TabularDatasetFactory.from_delimited_files(data_loc)
data = ds.to_pandas_dataframe().dropna()


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

I refered to the following link for the input parameters. 
https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py
<br>
Upon reading the various parameters I chose the following <br>
1. Primary_metric as accuracy as Accuracy was the objective function <br>
2. I set Max concurrent iterations as 10 to parallize the automl model building process<br>
3. I set experiment timeout to 25 minutes so it doesn't go on building the model for a really long time.<br>
4. I set the task as classification<br>
5. I provided the label column name<br>
6. The training dataset<br>
7. The compute target to be used for computing<br>
8. Cross_validation as 5<br>
I left the rest of the parameters as their default values since they made sense for my run.<br>

In [4]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 10,
    "primary_metric" : 'accuracy'}

computetarget = 'udacity'

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=computetarget,
    task='classification',
    training_data=ds,
    label_column_name='blueWins',   
    n_cross_validations=5,
    **automl_settings
)

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_7073e5c2-2d4e-490d-b6f2-593d408cc530',
 'target': 'udacity',
 'status': 'Completed',
 'startTimeUtc': '2021-02-06T21:32:41.621897Z',
 'endTimeUtc': '2021-02-06T22:05:16.411751Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'udacity',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-137671","workspace_name":"quick-starts-ws-137671","region":"southcentralus","compute_target":"udacity","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurizati

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: udacity-project,
Id: AutoML_7073e5c2-2d4e-490d-b6f2-593d408cc530_152,
Type: azureml.scriptrun,
Status: Completed)
recall_score_micro 0.7361071080817917
precision_score_macro 0.7360083004880879
f1_score_micro 0.7361071080817917
f1_score_macro 0.7359649730297262
average_precision_score_weighted 0.806556271787146
average_precision_score_micro 0.8060250385734833
weighted_accuracy 0.7361713911371649
recall_score_weighted 0.7361071080817917
average_precision_score_macro 0.8062971674279036
precision_score_micro 0.7361071080817917
precision_score_weighted 0.7362524491650624
norm_macro_recall 0.47208508837729735
accuracy 0.7361071080817917
AUC_macro 0.8091083894503107
log_loss 0.5347821264415756
matthews_correlation 0.4720508255754538
recall_score_macro 0.7360425441886487
f1_score_weighted 0.7361193573199138
balanced_accuracy 0.7360425441886487
AUC_micro 0.8090370588947993
AUC_weighted 0.8091083894503107
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_7073e5c2-2d4e-4

In [8]:
#TODO: Save the best model
best_model = best_run.register_model(model_path='./', model_name='automl_best_model',tags={'Training context':'Auto ML'},properties={'Accuracy': best_run_metrics['accuracy']})

print(best_model)

Model(workspace=Workspace.create(name='quick-starts-ws-137671', subscription_id='a0a76bad-11a1-4a2d-9887-97a29122c8ed', resource_group='aml-quickstarts-137671'), name=automl_best_model, id=automl_best_model:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.7361071080817917'})


In [9]:
# Saving the model in ONNX format. 

from azureml.automl.runtime.onnx_convert import OnnxConverter

best_run, onnx_model= remote_run.get_output(return_onnx_model=True)

#TODO: Save the best model
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_best_model.onnx")

OnnxConvertException: OnnxConvertException:
	Message: Requested an ONNX compatible model but the run has ONNX compatibility disabled.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Requested an ONNX compatible model but the run has ONNX compatibility disabled.",
        "target": "onnx_compatible",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        }
    }
}

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
# Download scoring file
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [16]:
# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_dependencies.yml')

In [25]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'type': "automl_classification"}, 
                                               description = 'LOL Game Winner Prediction')

aci_service_name = 'lolpredict'
print(aci_service_name)

aci_service = Model.deploy(ws, aci_service_name, [best_model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

print(aci_service.scoring_uri)

print(aci_service.swagger_uri)

lolpredict
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: cf75d17f-8c6f-48a9-b5cc-e92e8bdb7ec0
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\n\t1. Please check the logs for your container instance: lolpredict. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.\n\t2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\n\t3. You can also try to run image viennaglobal.azurecr.io/azureml/azureml_24c99391bdf28870da8705fecc8f2622 locally. Please refer to https://aka.ms/debugi

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: cf75d17f-8c6f-48a9-b5cc-e92e8bdb7ec0
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\n\t1. Please check the logs for your container instance: lolpredict. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.\n\t2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\n\t3. You can also try to run image viennaglobal.azurecr.io/azureml/azureml_24c99391bdf28870da8705fecc8f2622 locally. Please refer to https://ak

TODO: In the cell below, send a request to the web service you deployed to test it.

In [27]:
print(aci_service.get_logs())

None


TODO: In the cell below, print the logs of the web service and delete the service